# CrewAI

> Let the (agent) games begin


In [ ]:
# | default_exp crewai

## Tools

A tool is a skill or function that agents can utilize to perform actions.

### SearXNG

Let's define a tool that leverages our searchXNG installation


In [ ]:
# | export

import MediaMinder.search as search
import MediaMinder.scrape as scrape
import MediaMinder.html as html

from langchain.tools import tool

In [ ]:
# | export


class SearchTools:
    @tool("Search for news articles")
    def search_news(query: str) -> str:
        """Useful to search the web for news about a given topic and return relevant results."""
        results: list[search.SearchResult] = search.run_query(f"!news {query}", page=1)
        text_results = []
        for result in results:
            text_results.append(
                "\n".join(
                    [
                        f"Title: {result.title}",
                        f"URL: {result.url}",
                        f"Snippet: {result.content}",
                        "\n--------------------",
                    ]
                )
            )

        return "\n".join(text_results)

    @tool("Search for content on the web")
    def search_web(query: str) -> str:
        """Useful to search the web for content about a given topic and return relevant results."""
        results: list[search.SearchResult] = search.run_query(query, page=1)
        text_results = []
        for result in results:
            text_results.append(
                "\n".join(
                    [
                        f"Title: {result.title}",
                        f"URL: {result.url}",
                        f"Snippet: {result.content}",
                        "\n--------------------",
                    ]
                )
            )

        return "\n".join(text_results)

In [ ]:
SearchTools.search_news('"Continental AG"')

/Users/cameronpope/Library/Caches/pypoetry/virtualenvs/mediaminder-yyNIy1MV-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"Title: Carmakers flag difficult year on costs, muted EV demand\nURL: https://www.detroitnews.com/story/business/autos/2024/05/09/carmakers-flag-difficult-year-on-costs-muted-ev-demand/73630367007/\nSnippet: EV demand skid, inflation, muted economic growth in much of Europe and a protracted recovery in China are adding to the ...\n\n--------------------\nTitle: Conti remains optimistic despite 'weak' first quarter\nURL: https://www.tirebusiness.com/financial/continental-remains-optimistic-despite-weak-first-quarter\nSnippet: Continental AG had a weak start to 2024, but is optimistic about the rest of the year, including expecting increased demand ...\n\n--------------------\nTitle: Continental AG Slips To Q1 Loss On Weak Sales; Sees Earnings Growth Ahead, Confirms FY24 View\nURL: https://markets.businessinsider.com/news/stocks/continental-ag-slips-to-q1-loss-on-weak-sales-sees-earnings-growth-ahead-confirms-fy24-view-1033345093\nSnippet: German automotive parts maker Continental AG (CT

## Page Scraping

We also need a good way to scrape pages using headless browsers


In [ ]:
# | export


def _html_to_response(html_content: str, max_content_length: int = 4000) -> str:
    article_data: html.Article = html.extract_article(html_content)
    # truncate content if it is too long
    if len(article_data.content) > max_content_length:
        article_data.content = article_data.content[:max_content_length] + "..."
    return "\n".join(
        [
            f"Title: {article_data.metadata['title']}",
            f"Author: {article_data.metadata['author']}",
            f"Date: {article_data.metadata['date']}",
            f"Description: {article_data.metadata['description']}",
            f"Site Name: {article_data.metadata['sitename']}",
            f"URL: {article_data.metadata['url']}",
            "---",
            f"{article_data.content}",
        ]
    )


class BrowserTools:
    @tool("Scrape and extract articles from website content")
    def scrape_website(url: str) -> str:
        """Scrape a website and return the text content."""
        try:
            html_content = scrape.scrape_url(url)
            return _html_to_response(html_content)
        except Exception:
            # Return a stubbed response if the scraping fails
            return "\n".join(
                [
                    f"URL: {url}",
                    "---",
                    "Unable to scrape content from this URL",
                ]
            )

In [ ]:
test_content = open(
    "../test-data/Continental acquires mold specialist EMT for commercial and specialty tires.html"
).read()
_html_to_response(test_content)

'Title: Continental acquires mold specialist EMT for commercial and specialty tires\nAuthor: Christian Hinton\nDate: 2024-05-02\nDescription: With the acquisition of EMT, Continental said it can independently produce tire molds for all applications if required.\nSite Name: Tire Review Magazine\nURL: https://www.tirereview.com/continental-mold-specialist-acquisition/\n---\nContinental has acquired the mold manufacturing specialist EMT s.r.o. based in Púchov, Slovakia. All shares of Continental’s long-standing supplier were transferred to Continental effective April 2. The tire manufacturer had entered into a respective agreement with EMT’s shareholders, Dynamic Design (Romania) being the majority shareholder. All 107 employees, who have specialist knowledge in mold manufacturing, have been taken on by Continental, the company said. Further details of the agreement were not disclosed.\nEMT is a manufacturer of special tire molds, especially for commercial vehicle and specialty tires. Acc

## Agents

Let's define some agents that will perform key tasks for us around media analysis


In [ ]:
# | export

from crewai import Agent

In [ ]:
# | export


class MediaMinderAgents:
    def researcher(self, llm=None):
        return Agent(
            role="Searcher",
            goal="Find news stories mentioning a given company.",
            backstory="""Your job is to find news stories mentioning a given company and pass along the results to the reviewer for review.""",
            verbose=True,
            tools=[SearchTools.search_news],
            llm=llm,
        )
    
    def reviewer(self, llm=None):
        return Agent(
            role="Content Reviewer",
            goal="Review a list of news stories and pick the ones that are relevant.",
            backstory="""Look at the search result summary and eliminate any irrelevant articles. If you cannot tell from the search result summary then accept the article.""",
            verbose=True,
            llm=llm,
        )

    def analyst(self, llm=None):
        return Agent(
            role="Media Analyst",
            goal="Analyze a single news story and extract relevant information about a given company.",
            backstory="""The go-to person for analyzing news stories and extracting relevant information. 
            Your work is crucial for the company's decision-making process. First, ensure the article is really
            written about the given company. Identify all of the companies, products, key events and sentiments 
            mentioned in the article. You are working on a high-priority project for a major client.""",
            verbose=True,
            tools=[BrowserTools.scrape_website],
            llm=llm,
        )

## Tasks

Tasks are specific assignments completed by agents. They provide all necessary details for execution, such as a description, the agent responsible, required tools, and more, facilitating a wide range of action complexities.


In [ ]:
# | export

from crewai import Task

In [ ]:
# | export


class MediaMinderTasks:
    def research_news(self, agent, company_name: str) -> Task:
        return Task(
            description=f"Search for news articles about the given company. The customer is interested in recent news about: {company_name}.",
            agent=agent,
            expected_output="A list of articles about the given company, including the title, url and the search result snippet.",
        )
    
    def review_news(self, agent):
        return Task(
            description="Review the search results and eliminate any irrelevant articles. If you cannot tell from the search result summary then accept the article.",
            agent=agent,
            expected_output="A list of articles that are actually about the given company, including the title, url and the search result snippet.",
        )

    def analyze_news(self, agent):
        return Task(
            description="Analyze news articles about the given company and extract the URL, title, companies, products, events, people and sentiments mentioned in each article.",
            agent=agent,
            expected_output="A list containing the URL, title, the companies, products, events, people and sentiments mentioned in each article.",
        )

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

## Set up the Crew

The crew is a collection of agents focused on a given task


In [ ]:
# | export

from crewai import Crew, Process
from langchain_core.language_models.llms import LLM

In [ ]:
# | export


def create_crew(company: str, crew_llm: LLM) -> Crew:
    print("Creating MediaMinder Crew")
    print("  Company:", company)

    researcher = MediaMinderAgents().researcher(llm=crew_llm)
    reviewer = MediaMinderAgents().reviewer(llm=crew_llm)
    analyst = MediaMinderAgents().analyst(llm=crew_llm)

    research_news_task = MediaMinderTasks().research_news(researcher, company)
    review_news_task = MediaMinderTasks().review_news(reviewer)
    analyze_news_task = MediaMinderTasks().analyze_news(analyst)

    return Crew(
        name="MediaMinder Crew",
        agents=[researcher, analyst],
        tasks=[research_news_task, review_news_task, analyze_news_task],
        process=Process.sequential,  # Optional: Sequential task execution is default
        cache=True,
        memory=True,
        max_rpm=50,
        share_crew=True,
        llm=crew_llm,
    )

In [ ]:
crew = create_crew("Continental", llm)
crew.kickoff()

Creating MediaMinder Crew
  Company: Continental


> Entering new CrewAgentExecutor chain...
Thought: I need to search for recent news articles about the company Continental. Using the "Search for news articles" tool will help me gather the required information.

Action: Search for news articles
Action Input: {"query": "Continental recent news"} 

Title: The Pitch: MLS Rivalry Week presented by Continental Tire is Continues!
URL: https://www.mlssoccer.com/news/the-pitch-mls-rivalry-week-presented-by-continental-tire-is-continues
Snippet: Midweek action sees the Lionel Messi-led Inter Miami CF travel to Orlando City SC looking to claim two wins over their ...

--------------------
Title: ContiTPU Tubes are Continental’s Lightweight Response to 7 Layer Dip, from just 35g!
URL: https://www.msn.com/en-us/lifestyle/shopping/contitpu-tubes-are-continental-s-lightweight-response-to-7-layer-dip-from-just-35g/ar-BB1maWvJ?ocid=BingNewsSearch
Snippet: Continental lightweight 7-layer ContiTPU inne

2024-05-16 06:45:43,290 - 8501557952 - core.py-core:237 - WARNING: discarding data: None


 

URL: https://www.msn.com/en-us/lifestyle/shopping/contitpu-tubes-are-continental-s-lightweight-response-to-7-layer-dip-from-just-35g/ar-BB1maWvJ?ocid=BingNewsSearch
---
Unable to scrape content from this URL

It seems I encountered an issue with scraping the second article from MSN. I'll proceed by scraping the content of the third article instead.

Action: Scrape and extract articles from website content
Action Input: {"url": "https://www.computerweekly.com/news/366584870/Continental-implements-cross-domain-high-performance-computer-in-a-car"} 

Title: Continental implements cross-domain high-performance computer in a car | Computer Weekly
Author: Joe O’Halloran
Date: 2024-05-15
Description: Mobile platform provider Qualcomm teams with automotive firm Continental to implement a cross-domain high-performance computer (HPC) in a car based on cloud-based Automotive Edge Framework and Snapdragon system-on-chip.
Site Name: ComputerWeekly.com
URL: https://www.computerweekly.com/news/3665

"1. **Title:** The Pitch: MLS Rivalry Week presented by Continental Tire Continues!\n   - **URL:** [The Pitch: MLS Rivalry Week presented by Continental Tire Continues!](https://www.mlssoccer.com/news/the-pitch-mls-rivalry-week-presented-by-continental-tire-is-continues)\n   - **Companies Mentioned:** Continental, MLS, Inter Miami CF, Orlando City SC, New York City FC, New York Red Bulls, Toronto FC, CF Montréal, Colorado Rapids, Real Salt Lake\n   - **Products Mentioned:** Continental Tire\n   - **Key Events:** MLS Rivalry Week, Lionel Messi-led Inter Miami CF vs. Orlando City SC match, Hudson River Derby, Diego Chará honored for league record, St. Louis CITY SC supporters unveil tifo\n   - **People Mentioned:** Lionel Messi, Luis Suárez, Matías Rojas, Benjamin Cremaschi, Luis Muriel, Nico Lodeiro, Duncan McGuire, Julián Fernández, Agustín Ojeda, James Sands, Tayvon Gray, Malachi Jones, Daniel Edelman, John Tolkin, Frankie Amaya, Noah Eile\n   - **Sentiments:** Positive sentiments tow

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()